In [16]:
import requests
import json
import pandas as pd
import datetime as dt
import numpy as np
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
#pip install xmltodict
import xmltodict
import xml.etree.cElementTree as ET

In [17]:
cfdata = pd.read_csv('./data/cfdata.csv')

In [18]:
#cfdata = cfdata.drop(['Unnamed: 0'],axis = 1)
cfdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24541 entries, 0 to 24540
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   24541 non-null  int64  
 1   APVPERMYMD   24541 non-null  object 
 2   TRDSTATEGBN  24541 non-null  int64  
 3   TRDSTATENM   24541 non-null  object 
 4   DCBYMD       8624 non-null   object 
 5   SITEAREA     23486 non-null  object 
 6   BPLCNM       24541 non-null  object 
 7   X            24541 non-null  float64
 8   Y            24541 non-null  float64
 9   region       24541 non-null  object 
 10  ADDR         24541 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 2.1+ MB


In [19]:
cfdata.ADDR[:18]

0      서울특별시 강동구 양재대로87길 11 
1       서울특별시 강동구 진황도로27길 23
2        서울특별시 강동구 구천면로 428 
3         서울특별시 강동구 강동대로 205
4       서울특별시 강동구 풍성로63길 84 
5         서울특별시 강동구 올림픽로 660
6          서울특별시 강동구 진황도로 74
7       서울특별시 강동구 풍성로63길 79 
8        서울특별시 강동구 고덕로83길 28
9        서울특별시 강동구 천호대로 1016
10       서울특별시 강동구 양재대로 1355
11       서울특별시 강동구 동남로81길 76
12        서울특별시 강동구 천호대로 991
13       서울특별시 강동구 천호대로 1092
14    서울특별시 강동구 양재대로128길 50 
15       서울특별시 강동구 천호대로 1005
16      서울특별시 강동구 상암로51길 46 
17    서울특별시 강동구 천호대로 1138-18
Name: ADDR, dtype: object

In [41]:
url1 = 'https://dapi.kakao.com/v2/local/search/address.json?query='+ cfdata.ADDR[1]
headers = {"Authorization": "KakaoAK 6341d1e76477fbac498a469adc87108d"}
result1 = json.loads(str(requests.get(url1, headers=headers).text))
result1

{'documents': [{'address': {'address_name': '서울 강동구 천호동 167-73',
    'b_code': '1174010900',
    'h_code': '1174062000',
    'main_address_no': '167',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '강동구',
    'region_3depth_h_name': '천호3동',
    'region_3depth_name': '천호동',
    'sub_address_no': '73',
    'x': '127.131828859786',
    'y': '37.5394669965987'},
   'address_name': '서울 강동구 진황도로27길 23',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 강동구 진황도로27길 23',
    'building_name': '에스아이팰리스',
    'main_building_no': '23',
    'region_1depth_name': '서울',
    'region_2depth_name': '강동구',
    'region_3depth_name': '천호동',
    'road_name': '진황도로27길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.131828859786',
    'y': '37.5394669965987',
    'zone_no': '05333'},
   'x': '127.131828859786',
   'y': '37.5394669965987'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [21]:
all_pnu=[]


for i in range(len(cfdata.ADDR)):
    
    #카카오 API 주소검색
    
    url1 = 'https://dapi.kakao.com/v2/local/search/address.json?query='+ cfdata.ADDR[i]
    headers = {"Authorization": "KakaoAK 6341d1e76477fbac498a469adc87108d"}
    result1 = json.loads(str(requests.get(url1, headers=headers).text))
    
    if len(result1['documents']) == 0:    #주소검색이 되지 않는 주소일 시 건너뜀
        i += 1
        all_pnu.append('0')
    else:
        addinfo = result1['documents'][0]['address']
        
        #주소 정보 합쳐 pnu 만들기
        main_add = (4-len(addinfo['main_address_no'])) * '0' + addinfo['main_address_no']
        
        sub_add = (4-len(addinfo['sub_address_no'])) * '0' + addinfo['sub_address_no']
        
        mountain = str(list(map(lambda x: 0 if x == 'Y' else 1, addinfo['mountain_yn']))[0])
        
        pnu ='pnu=' + addinfo['b_code'] + mountain + main_add + sub_add

        all_pnu.append(pnu)

print(len(all_pnu))

24541


In [22]:
all_pnu.count('0')   #주소검색이 되지 않았던 가게 주소의 갯수

280

In [23]:
all_pnu[:25]

['pnu=1174010800104390011',
 'pnu=1174010900101670073',
 'pnu=1174010100103270001',
 'pnu=1174010800104480009',
 'pnu=1174010600104440000',
 'pnu=1174010900104270003',
 'pnu=1174010900101570003',
 'pnu=1174010600104420005',
 'pnu=1174010200101910005',
 'pnu=1174010800100480001',
 'pnu=1174010800104280005',
 'pnu=1174010200106450007',
 'pnu=1174010900104560009',
 'pnu=1174010800101940000',
 'pnu=1174010500103250001',
 'pnu=1174010900105720000',
 'pnu=1174010100103320003',
 'pnu=1174010800103790013',
 'pnu=1174010800103840000',
 'pnu=1174010900104080012',
 'pnu=1174010800105500005',
 'pnu=1174010800101200060',
 'pnu=1174010600104420011',
 'pnu=1174010300105010000',
 '0']

In [24]:
new_url=[]

# 만든 pnu로 공시지가 API 검색을 위한 url 리스트만들기

for i in range(len(all_pnu)):
    url = 'http://apis.data.go.kr/1611000/nsdi/ReferLandPriceService/wfs/getReferLandPriceWFS?'
    servicekey = 'ServiceKey=KbZnc4ks39Uh3gTHjC07NVnharI7QMm1IRiuIU09Sh8qAivpgKhrwDkBI95P0ujEpELuJ%2Bym4qoQihKvxy1Y1Q%3D%3D&'
    parameter = url+servicekey+all_pnu[i]
    new_url.append(parameter)
len(new_url)

24541

In [26]:
new_url[1]   #0~50중 0,17,36,38만 검색됨

'http://apis.data.go.kr/1611000/nsdi/ReferLandPriceService/wfs/getReferLandPriceWFS?ServiceKey=KbZnc4ks39Uh3gTHjC07NVnharI7QMm1IRiuIU09Sh8qAivpgKhrwDkBI95P0ujEpELuJ%2Bym4qoQihKvxy1Y1Q%3D%3D&pnu=1174010900101670073'

In [27]:
import urllib.request

In [39]:
request = urllib.request.urlopen(new_url[7])
request.get_method = lambda: 'GET'
response_body = request.read().decode('utf-8')

cc = xmltodict.parse(response_body) # return collections.OrderedDict
dd = json.loads(json.dumps(cc)) # return dict
#NSDI = dd['wfs:FeatureCollection']['gml:featureMember']['NSDI:F166']
dd

{'wfs:FeatureCollection': {'@xmlns:xs': 'http://www.w3.org/2001/XMLSchema',
  '@xmlns:wfs': 'http://www.opengis.net/wfs',
  '@xmlns:gml': 'http://www.opengis.net/gml',
  '@xmlns:NSDI': 'http://10.1.17.66:6080/arcgis/services/opend/EiosSpceServiceWFS/MapServer/WFSServer',
  '@xmlns:xlink': 'http://www.w3.org/1999/xlink',
  '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@timeStamp': '2021-02-09T17:31:20Z',
  '@numberOfFeatures': 'unknown',
  '@xsi:schemaLocation': 'http://www.opengis.net/wfs http://schemas.opengis.net/wfs/1.1.0/wfs.xsd http://www.opengis.net/gml http://schemas.opengis.net/gml/3.1.1/base/gml.xsd http://10.1.17.66:6080/arcgis/services/opend/EiosSpceServiceWFS/MapServer/WFSServer http://10.1.17.66:6080/arcgis/services/opendb/EiosSpceServiceWFS/MapServer/WFSServer?service=wfs%26version=1.1.0%26request=DescribeFeatureType'}}

In [360]:
PBLNTF_PCLND=[]
ROAD_SIDE_CODE=[]
PRPOS_AREA=[]
LNDPCL_AR=[]

for i in range(len(new_url)):
    request = urllib.request.urlopen(new_url[i])
    request.get_method = lambda: 'GET'
    response_body = request.read().decode('utf-8')
    type(response_body)
    response_body

    cc = xmltodict.parse(response_body) # return collections.OrderedDict
    dd = json.loads(json.dumps(cc)) # return dict
    NSDI = dd['wfs:FeatureCollection']['gml:featureMember']['NSDI:F166']
    print(NSDI)
    
    PBLNTF_PCLND.append(NSDI['NSDI:PBLNTF_PCLND'])
    ROAD_SIDE_CODE.append(NSDI['NSDI:ROAD_SIDE_CODE'])
    PRPOS_AREA.append(NSDI['NSDI:PRPOS_AREA'])
    LNDPCL_AR.append(NSDI['NSDI:LNDPCL_AR'])

{'@gml:id': 'F166.473862126', 'NSDI:SHAPE': {'gml:MultiSurface': {'gml:surfaceMember': {'gml:Polygon': {'gml:exterior': {'gml:LinearRing': {'gml:posList': '211800 447040.6799999997 211815 447036.2300000005 211817.3899999997 447031.8300000001 211813.2259999998 447017.7909999993 211798.5489999997 447022.1469999999 211786.8470000001 447025.6209999993 211792.0199999996 447043.0490000006 211800 447040.6799999997'}}}}}}, 'NSDI:PNU': '1174010800104390011', 'NSDI:LD_CPSG_CODE': '11740', 'NSDI:LD_EMD_LI_CODE': '10800', 'NSDI:REGSTR_SE_CODE': '1', 'NSDI:MNNM': '0439', 'NSDI:SLNO': '0011', 'NSDI:LNM_LNDCGR_SMBOL': '439-11대', 'NSDI:ISSU_CONFM_CODE': '1', 'NSDI:SRC_OBJECTID': '117310', 'NSDI:SRC_LD_CPSG_CODE': '11740', 'NSDI:PBLNTF_PCLND_STDR_YEAR': '2020', 'NSDI:PBLNTF_PCLND_STDR_MT': '01', 'NSDI:PBLNTF_PCLND': '7120000', 'NSDI:PSTYR_1_PBLNTF_PCLND': '6850000', 'NSDI:PSTYR_2_PBLNTF_PCLND': '6460000', 'NSDI:PSTYR_3_PBLNTF_PCLND': '6350000', 'NSDI:PSTYR_4_PBLNTF_PCLND': '6260000', 'NSDI:STD_LAND_AT'

KeyError: 'gml:featureMember'

Note: you may need to restart the kernel to use updated packages.
